In [14]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm


In [6]:
data = []
visited_urls = set()


def extract_text(element):
    if element.name == "p" or (
        element.name == "h2" and "wp-block-heading" in element.get("class", [])
    ):
        return element.text.replace("\n", " ").strip()
    else:
        return None


total_pages = 4

for x in tqdm(range(1, total_pages + 1), desc="Scraping pages"):
    response = requests.get(f"https://foodie.my/food/page/{x}")
    soup = BeautifulSoup(response.content, "html.parser")
    entries = soup.find_all("h3", attrs={"class": "post-title"})

    for entry in tqdm(entries, desc=f"Scraping entries on page {x}", leave=False):
        blog_url = entry.find("a").get("href")

        # Check if the URL has already been processed
        if blog_url not in visited_urls:
            blog_title = entry.text.strip()

            page = requests.get(blog_url)
            blog_soup = BeautifulSoup(page.text, "html.parser")

            body = blog_soup.find(
                "div", attrs={"class": "entry-content entry clearfix"}
            )

            texts = [extract_text(tag) for tag in body.find_all(["p", "h2"])]
            content_text = "\n".join(filter(None, texts))

            data.append({"url": blog_url, "title": blog_title, "body": content_text})

            # Add the URL to the set to mark it as processed
            visited_urls.add(blog_url)

Scraping pages: 100%|██████████| 4/4 [00:30<00:00,  7.55s/it]


In [11]:
with open("foodie-my-food.json", "w") as json_file:
    json.dump(data, json_file, indent=2)

In [13]:
# import pandas as pd

# # open foodie-my-food.json as csv
# df = pd.read_json("foodie-my-food.json")
# df